In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from datetime import datetime
from datetime import timedelta
from tqdm import tqdm

# Load Data

In [75]:
train_data = pd.read_csv('dataset/Google_Stock_Train.csv', parse_dates=True, index_col='Date', header=0)
print(train_data.shape)
train_data.head()

(3272, 6)


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2010-01-04,15.689439,15.753504,15.621622,15.684434,15.684434,78169752
2010-01-05,15.695195,15.711712,15.554054,15.615365,15.615365,120067812
2010-01-06,15.662162,15.662162,15.174174,15.221722,15.221722,158988852
2010-01-07,15.250250,15.265265,14.831081,14.867367,14.867367,256315428
2010-01-08,14.814815,15.096346,14.742492,15.065566,15.065566,188783028


In [76]:
test_data = pd.read_csv('dataset/Google_Stock_Test.csv', parse_dates=True, index_col='Date', header=0)
print(test_data.shape)
test_data.head()

(143, 6)


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2023-01-03,89.589996,91.050003,88.519997,89.120003,89.120003,28131200
2023-01-04,90.349998,90.650002,87.269997,88.080002,88.080002,34854800
2023-01-05,87.470001,87.570000,85.900002,86.199997,86.199997,27194400
2023-01-06,86.790001,87.690002,84.860001,87.339996,87.339996,41381500
2023-01-09,88.360001,90.050003,87.860001,88.019997,88.019997,29003900


In [77]:
full_data = pd.concat((train_data, test_data), ignore_index=True)
full_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3415 entries, 0 to 3414
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       3415 non-null   float64
 1   High       3415 non-null   float64
 2   Low        3415 non-null   float64
 3   Close      3415 non-null   float64
 4   Adj Close  3415 non-null   float64
 5   Volume     3415 non-null   int64  
dtypes: float64(5), int64(1)
memory usage: 160.2 KB


## Normalizing Data

In [6]:
def normalize_data(data, min, max):
    data = data - min
    data = data/max
    return data

data_norm = normalize_data(full_data, full_data.min(axis=0), full_data.max(axis=0))
data_norm.head()

,Open,High,Low,Close,Adj Close,Volume
0,0.031211,0.030917,0.032035,0.031846,0.031846,0.116235
1,0.031249,0.030642,0.031581,0.031385,0.031385,0.186962
2,0.031031,0.030315,0.029030,0.028758,0.028758,0.252662
3,0.028308,0.027696,0.026726,0.026393,0.026393,0.416954
4,0.025429,0.026581,0.026131,0.027716,0.027716,0.302956


In [7]:
x_train = data_norm[:train_data.shape[0]]
x_test = data_norm[train_data.shape[0]:]

# Windowed Dataset

In [8]:
n_features = full_data.shape[1]
n_past = 24
n_future = 24 
shift = 1
batch_size = 32

In [9]:
def windowed_dataset(series, batch_size, n_past, n_future, shift):
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(size=n_past + n_future, shift=shift, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(n_past+n_future))
    ds = ds.shuffle(1000)
    ds = ds.map(lambda w: (w[:-n_past], w[-n_past:, :1]))
    ds = ds.batch(batch_size).prefetch(1)
    return ds

train_set = windowed_dataset(x_train, batch_size, n_past, n_future, shift)
test_set = windowed_dataset(x_train, batch_size, n_past, n_future, shift)

# Models

In [24]:
result = pd.DataFrame(columns=['Model','Loss', 'Accuracy'])
result

,Model,Loss,Accuracy


## LSTM

In [20]:
lstm = tf.keras.models.Sequential([
        tf.keras.layers.Input(shape=(n_past,n_features)),
        tf.keras.layers.LSTM(128, activation='relu'),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(n_future * n_features, activation='relu'),
        tf.keras.layers.Reshape((n_future, n_features)),
    ])
    
lstm.compile(
    optimizer=tf.keras.optimizers.SGD(learning_rate=0.001, momentum=0.9),
    loss='mae',
    metrics=['accuracy']
    )

lstm.fit(train_set, epochs=20, validation_data=test_set)

Epoch 1/20
101/101 ━━━━━━━━━━━━━━━━━━━━ 3s 20ms/step - accuracy: 9.7305e-05 - loss: 0.1273 - val_accuracy: 6.4599e-05 - val_loss: 0.2381
Epoch 2/20
101/101 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - accuracy: 9.6877e-05 - loss: 0.1198 - val_accuracy: 7.7519e-05 - val_loss: 0.2301
Epoch 3/20
101/101 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - accuracy: 1.4970e-04 - loss: 0.1146 - val_accuracy: 6.4599e-05 - val_loss: 0.2225
Epoch 4/20
101/101 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - accuracy: 1.3124e-04 - loss: 0.1108 - val_accuracy: 6.4599e-05 - val_loss: 0.2150
Epoch 5/20
101/101 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - accuracy: 8.3697e-05 - loss: 0.1085 - val_accuracy: 6.4599e-05 - val_loss: 0.2071
Epoch 6/20
101/101 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - accuracy: 1.0008e-04 - loss: 0.1063 - val_accuracy: 6.4599e-05 - val_loss: 0.1982
Epoch 7/20
101/101 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - accuracy: 2.2695e-04 - loss: 0.1054 - val_accuracy: 7.7519e-05 - val_loss: 0.1884
Epoch 8/20
101/101 ━━━━━━━━━━━━━━━━━━━━ 2

In [27]:
accuracy, loss = lstm.evaluate(test_set)
result.loc[0] = ['LSTM', accuracy, loss]
result

101/101 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 1.1586e-04 - loss: 0.0587


d:\2. NGODINGS\AI ML DATA\Project\Stock Trading Simulation\.venv\lib\site-packages\keras\src\trainers\epoch_iterator.py:151: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


,Model,Loss,Accuracy
0,LSTM,0.105878,0.000052
1,LSTM,0.105878,0.000052


## Bidirectional LSTM

In [39]:
bilstm = tf.keras.models.Sequential([
        tf.keras.layers.Input(shape=(n_past,n_features)),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True)),
        tf.keras.layers.LSTM(64, activation='relu'),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(n_future * n_features, activation='relu'),
        tf.keras.layers.Reshape((n_future, n_features)),
    ])
    
bilstm.compile(
    optimizer=tf.keras.optimizers.SGD(learning_rate=0.001, momentum=0.9),
    loss='mae',
    metrics=['accuracy']
    )

bilstm.fit(train_set, epochs=20, validation_data=test_set)

Epoch 1/20
     99/Unknown 5s 24ms/step - accuracy: 1.3497e-04 - loss: 0.1274

d:\2. NGODINGS\AI ML DATA\Project\Stock Trading Simulation\.venv\lib\site-packages\keras\src\trainers\epoch_iterator.py:151: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


101/101 ━━━━━━━━━━━━━━━━━━━━ 7s 39ms/step - accuracy: 1.3443e-04 - loss: 0.1308 - val_accuracy: 1.2920e-04 - val_loss: 0.2399
Epoch 2/20
101/101 ━━━━━━━━━━━━━━━━━━━━ 4s 35ms/step - accuracy: 1.8893e-04 - loss: 0.1205 - val_accuracy: 1.2920e-04 - val_loss: 0.2314
Epoch 3/20
101/101 ━━━━━━━━━━━━━━━━━━━━ 4s 38ms/step - accuracy: 2.4395e-04 - loss: 0.1158 - val_accuracy: 1.1628e-04 - val_loss: 0.2237
Epoch 4/20
101/101 ━━━━━━━━━━━━━━━━━━━━ 4s 38ms/step - accuracy: 1.8596e-04 - loss: 0.1129 - val_accuracy: 1.0336e-04 - val_loss: 0.2159
Epoch 5/20
101/101 ━━━━━━━━━━━━━━━━━━━━ 4s 37ms/step - accuracy: 1.0526e-04 - loss: 0.1100 - val_accuracy: 1.0336e-04 - val_loss: 0.2081
Epoch 6/20
101/101 ━━━━━━━━━━━━━━━━━━━━ 4s 35ms/step - accuracy: 1.6270e-04 - loss: 0.1093 - val_accuracy: 1.0336e-04 - val_loss: 0.1997
Epoch 7/20
101/101 ━━━━━━━━━━━━━━━━━━━━ 4s 39ms/step - accuracy: 1.9776e-04 - loss: 0.1093 - val_accuracy: 1.0336e-04 - val_loss: 0.1904
Epoch 8/20
101/101 ━━━━━━━━━━━━━━━━━━━━ 4s 38ms/step

In [43]:
accuracy, loss = bilstm.evaluate(test_set)
result.loc[1] = ['BiLSTM', accuracy, loss]
result

101/101 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 5.1153e-05 - loss: 0.0589


d:\2. NGODINGS\AI ML DATA\Project\Stock Trading Simulation\.venv\lib\site-packages\keras\src\trainers\epoch_iterator.py:151: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


,Model,Loss,Accuracy
0,LSTM,0.105878,0.000052
1,BiLSTM,0.108495,0.000039
2,GRU,0.097550,0.000052


## GRU

In [53]:
gru = tf.keras.models.Sequential([
        tf.keras.layers.Input(shape=(n_past,n_features)),
        tf.keras.layers.GRU(128, return_sequences=True, dropout=0.3),
        tf.keras.layers.GRU(64),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(n_future * n_features, activation='relu'),
        tf.keras.layers.Reshape((n_future, n_features)),
    ])
    
gru.compile(
    optimizer=tf.keras.optimizers.SGD(learning_rate=0.001, momentum=0.9),
    loss='mae',
    metrics=['accuracy']
    )

gru.fit(train_set, epochs=20, validation_data=test_set)

Epoch 1/20
     99/Unknown 5s 20ms/step - accuracy: 1.1130e-04 - loss: 0.1243

d:\2. NGODINGS\AI ML DATA\Project\Stock Trading Simulation\.venv\lib\site-packages\keras\src\trainers\epoch_iterator.py:151: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


101/101 ━━━━━━━━━━━━━━━━━━━━ 6s 32ms/step - accuracy: 1.0955e-04 - loss: 0.1277 - val_accuracy: 1.2920e-05 - val_loss: 0.2364
Epoch 2/20
101/101 ━━━━━━━━━━━━━━━━━━━━ 3s 28ms/step - accuracy: 2.4114e-05 - loss: 0.1177 - val_accuracy: 1.2920e-05 - val_loss: 0.2250
Epoch 3/20
101/101 ━━━━━━━━━━━━━━━━━━━━ 3s 29ms/step - accuracy: 2.4114e-05 - loss: 0.1131 - val_accuracy: 6.4599e-05 - val_loss: 0.2142
Epoch 4/20
101/101 ━━━━━━━━━━━━━━━━━━━━ 3s 26ms/step - accuracy: 6.6243e-05 - loss: 0.1085 - val_accuracy: 5.1680e-05 - val_loss: 0.2046
Epoch 5/20
101/101 ━━━━━━━━━━━━━━━━━━━━ 3s 30ms/step - accuracy: 1.0609e-04 - loss: 0.1064 - val_accuracy: 3.8760e-05 - val_loss: 0.1963
Epoch 6/20
101/101 ━━━━━━━━━━━━━━━━━━━━ 3s 29ms/step - accuracy: 1.6006e-04 - loss: 0.1082 - val_accuracy: 5.1680e-05 - val_loss: 0.1889
Epoch 7/20
101/101 ━━━━━━━━━━━━━━━━━━━━ 3s 29ms/step - accuracy: 1.2543e-04 - loss: 0.1086 - val_accuracy: 6.4599e-05 - val_loss: 0.1822
Epoch 8/20
101/101 ━━━━━━━━━━━━━━━━━━━━ 3s 30ms/step

In [44]:
accuracy, loss = gru.evaluate(test_set)
result.loc[2] = ['GRU', accuracy, loss]
result

101/101 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 8.6725e-05 - loss: 0.0536


d:\2. NGODINGS\AI ML DATA\Project\Stock Trading Simulation\.venv\lib\site-packages\keras\src\trainers\epoch_iterator.py:151: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


,Model,Loss,Accuracy
0,LSTM,0.105878,0.000052
1,BiLSTM,0.108495,0.000039
2,GRU,0.097550,0.000052
